In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_two_column_with_rank_11_withoutDT.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  127.425882     7.040131
1  233.628927    12.907725
2  134.062123     7.406775
3  245.587090    13.568399
4  364.485990    20.137424


MAP THE LOAD OF MEANHLY AVERGAE WITH THE FEEDER

In [ ]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


COMMULATIVE DIFFERENCE FOR MONTHLY AVERAGE LOAD OF SCADA


In [3]:
import pandas as pd

# File paths
feeder_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv"
monthly_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv"
output_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv"


# Load data
feeder_df = pd.read_csv(feeder_path)
monthly_df = pd.read_csv(monthly_path)

# Merge on FEEDER_ID <-> SWNO
merged = feeder_df.merge(monthly_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

month_cols = [f'Month_{i:02d}' for i in range(1, 13)]

result_rows = []

for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
    group = group.copy()
    group['__orig_idx'] = group.index
    ranks_in_order = group['RANK'].drop_duplicates().tolist()

    dt_load_sums = {
        rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
        for rank in ranks_in_order
    }

    prev_cum = {m: None for m in month_cols}

    for rank in ranks_in_order:
        this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
        first = True
        for idx, row in this_rank_rows.iterrows():
            base = {col: row[col] for col in feeder_df.columns}
            for m in month_cols:
                month_val = row.get(m, None)
                if first:
                    # Only for the first row of this rank
                    if rank == ranks_in_order[0]:
                        cum_val = month_val - dt_load_sums[rank] if pd.notna(month_val) else None
                    else:
                        prev = prev_cum[m]
                        cum_val = prev - dt_load_sums[rank] if prev is not None else None
                    prev_cum[m] = cum_val
                else:
                    cum_val = None  # For subsequent rows at same rank, leave blank
                base[f"{m}_cumulative_diff"] = cum_val
            result_rows.append(base)
            first = False

result_df = pd.DataFrame(result_rows)
output_cols = list(feeder_df.columns) + [f"{m}_cumulative_diff" for m in month_cols]
result_df.to_csv(output_path, index=False, columns=output_cols)
print(f"Saved to {output_path}")


Saved to /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv


AFTER THE RANKING AND USE ONLY MAIN CHAIN DONT USING SUBCHAIN

In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  616.320000    34.050959
1  674.080488    37.242158
2  473.442574    26.157148
3  148.855367     8.224085
4  400.698507    22.138124


In [3]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


In [1]:
import pandas as pd

# File paths (change as needed)
feeder_path = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv"
daily_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2025.csv"
daily_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2024.csv"
output_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv"
output_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv"


In [2]:
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()


In [3]:
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    output_cols = list(feeder_df.columns) + [f"{d}_cumulative_diff" for d in day_cols]
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# Process each year
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv


In [4]:
import pandas as pd

# --- Identify only main-chain feeders (no sub-branches) ---
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()

# --- Processing function ---
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in main_chain_feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    # --- Add FROM_SWITCH and TO_SWITCH columns ---
    if "FROM_TO" in result_df.columns:
        from_to_split = result_df["FROM_TO"].str.split("-", n=1, expand=True)
        result_df["FROM_SWITCH"] = from_to_split[0]
        result_df["TO_SWITCH"] = from_to_split[1]
        cols = list(result_df.columns)
        i = cols.index("FROM_TO") + 1
        for col in ["FROM_SWITCH", "TO_SWITCH"]:
            cols.insert(i, cols.pop(cols.index(col)))
            i += 1
        result_df = result_df[cols]
    # --- Save ---
    output_cols = list(result_df.columns)  # Save all columns, now includes FROM_SWITCH and TO_SWITCH
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# --- Process each year ---
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv


NEW LOGIC TO CALCULATE THE LOAD

In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Node & Section loads (bottom-up) + cumulative section loads.
Data source for feeder monthly averages: monthly_SWNO_matrix_11KV.csv (SWNO == FEEDER_ID)

Adds to output:
  - FRACTION, SUM_DT_LOAD_CAL
  - feeder_2024_01.._12
  - node_load_2024_01.._12
  - sec_load_2024_01.._12
  - sec_cum_2024_01.._12

Writes: fedder_11KV_with_all_monthly_loads.csv
"""

from __future__ import annotations
import os, math
from pathlib import Path
from typing import List, Optional, Dict, Tuple

import numpy as np
import pandas as pd

# ── Paths ───────────────────────────────────────────────────────────────────
BASE      = "/media/sagark24/New Volume/MERGE CDIS"
IN_FEDDER = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")
IN_MATRIX = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/monthly_SWNO_matrix_11KV.csv")
OUT_FILE  = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv")

# ── Month keys/tags ─────────────────────────────────────────────────────────
YEAR = 2024
MONTH_KEYS = [f"Month_{i:02d}" for i in range(1, 13)]      # matrix column names
MONTH_TAGS = [f"{YEAR}-{i:02d}" for i in range(1, 13)]      # long format tags
NODE_COLS  = [f"node_load_{YEAR}_{i:02d}" for i in range(1, 13)]
SEC_COLS   = [f"sec_load_{YEAR}_{i:02d}"  for i in range(1, 13)]
CUM_COLS   = [f"sec_cum_{YEAR}_{i:02d}"   for i in range(1, 13)]
FEED_COLS  = [f"feeder_{YEAR}_{i:02d}"    for i in range(1, 13)]

# ── Helpers ─────────────────────────────────────────────────────────────────
def norm_id_series(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip()

def make_feed_month_from_matrix(mat: pd.DataFrame) -> pd.DataFrame:
    """Return FEEDER_ID|MONTH|FEEDER_LOAD_AVG (complete Jan..Dec per feeder)."""
    need = {"SWNO", *MONTH_KEYS}
    miss = [c for c in need if c not in mat.columns]
    if miss:
        raise RuntimeError(f"monthly matrix missing columns: {miss}")

    m = mat.copy()
    m["FEEDER_ID"] = norm_id_series(m["SWNO"])
    for c in MONTH_KEYS:
        m[c] = pd.to_numeric(m[c], errors="coerce").fillna(0.0)

    long = (m.melt(id_vars=["FEEDER_ID"], value_vars=MONTH_KEYS,
                   var_name="MONTH_KEY", value_name="FEEDER_LOAD_AVG")
              .assign(MONTH=lambda d: d["MONTH_KEY"].str.replace("Month_", f"{YEAR}-", regex=False))
              .drop(columns=["MONTH_KEY"]))
    complete = (pd.MultiIndex.from_product([m["FEEDER_ID"].unique(), MONTH_TAGS],
                                           names=["FEEDER_ID","MONTH"]).to_frame(index=False))
    long = complete.merge(long, on=["FEEDER_ID","MONTH"], how="left").fillna({"FEEDER_LOAD_AVG": 0.0})
    return long

def parent_rank(r: str) -> Optional[str]:
    parts = str(r).split(".")
    return ".".join(parts[:-1]) if len(parts) > 1 else None

# def compute_section_bottom_up(df_f: pd.DataFrame, node_cols: List[str]) -> pd.DataFrame:
#     """
#     Bottom-up section aggregation for a single feeder.
#     Input df_f must have columns: FEEDER_ID, RANK, DESTINATION_LOCATION, node_cols...
#     Returns FEEDER_ID, RANK, DESTINATION_LOCATION, sec_cols..., cum_cols...
#     """
#     cols_keep = ["FEEDER_ID","RANK","DESTINATION_LOCATION"] + node_cols
#     sec = df_f[cols_keep].drop_duplicates(["FEEDER_ID","RANK","DESTINATION_LOCATION"]).copy()
#     sec[node_cols] = sec[node_cols].fillna(0.0)
#     sec["RANK"] = sec["RANK"].astype(str)

#     # index maps
#     idx_by_rank: Dict[str, int] = {r: i for i, r in enumerate(sec["RANK"].tolist())}
#     depth = sec["RANK"].str.count(r"\.").to_numpy() + 1

#     # matrices
#     M = sec[node_cols].to_numpy(dtype=float)  # self node loads (n x 12)
#     T = M.copy()                               # totals → will accumulate children

#     # process deepest → root
#     for i in np.argsort(-depth):
#         pr = parent_rank(sec.iat[i, sec.columns.get_loc("RANK")])
#         if pr and pr in idx_by_rank:
#             T[idx_by_rank[pr], :] += T[i, :]

#     # build outputs
#     out = sec[["FEEDER_ID","RANK","DESTINATION_LOCATION"]].copy()
#     # sec monthly
#     for j, col in enumerate(SEC_COLS):
#         out[col] = T[:, j]
#     # cumulative across months (axis=1, Jan→Dec)
#     cum = np.cumsum(T, axis=1)
#     for j, col in enumerate(CUM_COLS):
#         out[col] = cum[:, j]
#     return out
def compute_section_bottom_up(df_f: pd.DataFrame, node_cols: list[str]) -> pd.DataFrame:
    """
    LEAF-ONLY aggregation:
      - Leaves contribute their node_load_*.
      - Non-leaves contribute 0 as node load.
      - For each parent, sec_load_* = sum of children's totals.
      - Propagates upward (child → parent → grandparent...) for every month at once.

    Input df_f must have: FEEDER_ID, RANK, DESTINATION_LOCATION, and node_cols.
    Returns: FEEDER_ID, RANK, DESTINATION_LOCATION, sec_load_YYYY_MM (and can add cum if you like).
    """
    # 1) Unique rows per section; ensure numeric matrix for all months
    cols_keep = ["FEEDER_ID", "RANK", "DESTINATION_LOCATION"] + node_cols
    sec = (df_f[cols_keep]
           .drop_duplicates(["FEEDER_ID","RANK","DESTINATION_LOCATION"])
           .copy())
    sec[node_cols] = sec[node_cols].fillna(0.0)
    sec["RANK"] = sec["RANK"].astype(str)

    # 2) Build children map to identify leaves (nodes with no children)
    ranks = sec["RANK"].tolist()
    # parent of "a.b.c" is "a.b"; root has no parent
    def parent_rank(r: str) -> str | None:
        parts = r.split(".")
        return ".".join(parts[:-1]) if len(parts) > 1 else None

    # children lists
    children = {r: [] for r in ranks}
    for r in ranks:
        p = parent_rank(r)
        if p in children:
            children[p].append(r)

    is_leaf = np.array([len(children[r]) == 0 for r in ranks])

    # 3) Node matrix M: keep node loads ONLY for leaves, zero for non-leaves
    M = sec[node_cols].to_numpy(dtype=float)
    M[~is_leaf, :] = 0.0  # <-- key change vs previous version

    # This will hold subtree totals; initialize with the leaf-only node matrix
    T = M.copy()

    # 4) Accumulate bottom-up: add each child’s totals into its parent, deepest → root
    # Depth = number of dots + 1
    depth = sec["RANK"].str.count(r"\.").to_numpy() + 1
    idx_by_rank = {r: i for i, r in enumerate(ranks)}

    for i in np.argsort(-depth):  # deepest first
        r = ranks[i]
        p = parent_rank(r)
        if p is not None and p in idx_by_rank:
            T[idx_by_rank[p], :] += T[i, :]

    # 5) Package output: rename node_load_YYYY_MM → sec_load_YYYY_MM
    out = sec[["FEEDER_ID","RANK","DESTINATION_LOCATION"]].copy()
    for j, ncol in enumerate(node_cols):
        sec_col = f"sec_load_{ncol.split('_', 2)[-1]}"  # node_load_2024_01 -> sec_load_2024_01
        out[sec_col] = T[:, j]

    # (Optional) cumulative across months per section:
    # cum = np.cumsum(T, axis=1)
    # for j, ncol in enumerate(node_cols):
    #     out[f"sec_cum_{ncol.split('_', 2)[-1]}"] = cum[:, j]

    return out

# ── Main ────────────────────────────────────────────────────────────────────
def main():
    if not IN_FEDDER.exists():
        raise FileNotFoundError(f"Feeder file not found: {IN_FEDDER}")
    if not IN_MATRIX.exists():
        raise FileNotFoundError(f"Monthly matrix not found: {IN_MATRIX}")

    # 1) Base trace table
    df = pd.read_csv(IN_FEDDER, low_memory=False)
    if "FEEDER_ID" not in df.columns or "RANK" not in df.columns:
        raise RuntimeError("Input must have FEEDER_ID and RANK")
    df["FEEDER_ID"] = norm_id_series(df["FEEDER_ID"])
    if "LOCATION" not in df.columns:
        if "DESTINATION_LOCATION" in df.columns:
            df["LOCATION"] = df["DESTINATION_LOCATION"]
        else:
            raise RuntimeError("Need LOCATION or DESTINATION_LOCATION")

    # 2) Ensure DT_LOAD_CAL (once)
    if "DT_LOAD_CAL" not in df.columns:
        den = 11 * 0.95 * math.sqrt(3.0)
        df["DT_LOAD_CAL"] = pd.to_numeric(df.get("DT_LOAD", np.nan), errors="coerce") / den

    # 3) Fractions per feeder (dedup by LOCATION)
    uniq = (df[["FEEDER_ID","LOCATION","DT_LOAD_CAL"]]
            .dropna(subset=["LOCATION"])
            .groupby(["FEEDER_ID","LOCATION"], as_index=False)
            .agg(DT_LOAD_CAL=("DT_LOAD_CAL","sum")))  # use 'sum' if parallel DTs must add
    sum_by_fid = uniq.groupby("FEEDER_ID")["DT_LOAD_CAL"].sum().rename("SUM_DT_LOAD_CAL")
    uniq = uniq.merge(sum_by_fid, on="FEEDER_ID", how="left")
    uniq["FRACTION"] = uniq["DT_LOAD_CAL"] / uniq["SUM_DT_LOAD_CAL"].replace(0, np.nan)

    # Merge only needed fields (avoid *_x/_y)
    loc_key = "DESTINATION_LOCATION" if "DESTINATION_LOCATION" in df.columns else "LOCATION"
    df = df.merge(
        uniq.rename(columns={"LOCATION": loc_key})[["FEEDER_ID", loc_key, "SUM_DT_LOAD_CAL", "FRACTION"]],
        on=["FEEDER_ID", loc_key],
        how="left",
        validate="many_to_one"
    )

    # 4) Feeder monthly averages from matrix
    matrix = pd.read_csv(IN_MATRIX, low_memory=False)
    feed_month = make_feed_month_from_matrix(matrix)
    feed_month["FEEDER_ID"] = norm_id_series(feed_month["FEEDER_ID"])

    feed_wide = (feed_month.pivot_table(index="FEEDER_ID", columns="MONTH",
                                        values="FEEDER_LOAD_AVG", aggfunc="mean", fill_value=0.0)
                              .reindex(columns=MONTH_TAGS, fill_value=0.0)
                              .reset_index())
    feed_wide.columns = ["FEEDER_ID"] + FEED_COLS
    df = df.merge(feed_wide, on="FEEDER_ID", how="left")

    # 5) Node monthly loads in one shot (vectorized)
    out_nodes = (uniq[["FEEDER_ID","LOCATION","FRACTION"]]
                 .merge(feed_month, on="FEEDER_ID", how="left"))
    out_nodes["CORRECTED_DT_LOAD"] = out_nodes["FRACTION"].fillna(0.0) * out_nodes["FEEDER_LOAD_AVG"].fillna(0.0)

    node_wide = (out_nodes.pivot_table(index=["FEEDER_ID","LOCATION"], columns="MONTH",
                                       values="CORRECTED_DT_LOAD", aggfunc="sum", fill_value=0.0)
                           .reindex(columns=MONTH_TAGS, fill_value=0.0)
                           .reset_index())
    node_wide.columns = ["FEEDER_ID","LOCATION"] + NODE_COLS
    # attach node loads to each trace row (DESTINATION_LOCATION)
    df = df.merge(node_wide.rename(columns={"LOCATION": loc_key}),
                  on=["FEEDER_ID", loc_key], how="left")

    # 6) Section loads (bottom-up) for every month simultaneously + cumulative
    sec_parts: List[pd.DataFrame] = []
    need = ["FEEDER_ID","RANK","DESTINATION_LOCATION"] + NODE_COLS
    missing = [c for c in need if c not in df.columns]
    if missing:
        raise RuntimeError(f"Missing columns for section aggregation: {missing}")

    for fid, grp in df[need].groupby("FEEDER_ID", sort=False):
        sec_f = compute_section_bottom_up(grp, NODE_COLS)   # returns sec_load_* and sec_cum_*
        sec_parts.append(sec_f)

    sec_all = pd.concat(sec_parts, ignore_index=True) if sec_parts else pd.DataFrame(
        columns=["FEEDER_ID","RANK","DESTINATION_LOCATION"] + SEC_COLS + CUM_COLS
    )

    df = df.merge(sec_all, on=["FEEDER_ID","RANK","DESTINATION_LOCATION"], how="left")

    # 7) Save
    df.to_csv(OUT_FILE, index=False)
    print(f"✓ Wrote {OUT_FILE}")

    # peek: shows node, section, cumulative for Jan
    peek = ["FEEDER_ID","RANK",loc_key,"node_load_2024_01","sec_load_2024_01","sec_cum_2024_01"]
    print(df[[c for c in peek if c in df.columns]].head(12).to_string(index=False))

if __name__ == "__main__":
    main()


✓ Wrote /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv
FEEDER_ID RANK   DESTINATION_LOCATION  node_load_2024_01  sec_load_2024_01
    10205    1 1S-MH-MU-ZSC-CL09-2382           7.187435          7.187435
    10205    2 1S-MH-MU-ZSC-CL09-2383           7.861029          7.861029
    10205    3 1S-MH-MU-ZSC-CL09-2723           5.521219          8.892494
    10205  3.1 1S-MH-MU-ZSC-CL09-2366           1.735930          1.735930
    10205  3.2 1S-MH-MU-ZSC-CL09-2348           4.672888          4.672888
    10205  3.3 1S-MH-MU-ZSC-CL09-2019           2.483676          2.483676
    10205    4 1S-MH-MU-ZSC-CL09-2074           2.414947          8.299723
    10205  4.1 1S-MH-MU-ZSC-CL06-3404           0.000000          0.000000
    10205  4.2 1S-MH-MU-ZSC-CL06-3489           2.368178          2.368178
    10205  4.3 1S-MH-MU-ZSC-CL06-2088           5.931545          5.931545
    10205    5 1S-MH-MU-ZSC-CL09-3373    

In [11]:
df= pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv")
col = df['FEEDER_ID'].unique()
print(len(col))

1099
